In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"oliverli0414","key":"19df8e5218b8495a8c1ea362134c4a27"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c home-credit-default-risk
!unzip home-credit-default-risk.zip

 99% 680M/688M [00:44<00:00, 72.5MB/s]
100% 688M/688M [00:44<00:00, 16.1MB/s]
Archive:  home-credit-default-risk.zip
  inflating: HomeCredit_columns_description.csv  
  inflating: POS_CASH_balance.csv    
  inflating: application_test.csv    
  inflating: application_train.csv   
  inflating: bureau.csv              
  inflating: bureau_balance.csv      
  inflating: credit_card_balance.csv  
  inflating: installments_payments.csv  
  inflating: previous_application.csv  
  inflating: sample_submission.csv   


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
application_train = pd.read_csv('application_train.csv')[['SK_ID_CURR','TARGET']]
credit_card_balance = pd.read_csv('credit_card_balance.csv')

In [ ]:
print(credit_card_balance.isna().sum())
credit_card_balance[['AMT_DRAWINGS_ATM_CURRENT','AMT_DRAWINGS_CURRENT','AMT_DRAWINGS_OTHER_CURRENT','AMT_DRAWINGS_POS_CURRENT']]=credit_card_balance[['AMT_DRAWINGS_ATM_CURRENT','AMT_DRAWINGS_CURRENT','AMT_DRAWINGS_OTHER_CURRENT','AMT_DRAWINGS_POS_CURRENT']].fillna(0)
credit_card_balance[['CNT_DRAWINGS_ATM_CURRENT','CNT_DRAWINGS_CURRENT','CNT_DRAWINGS_OTHER_CURRENT','CNT_DRAWINGS_POS_CURRENT']]=credit_card_balance[['CNT_DRAWINGS_ATM_CURRENT','CNT_DRAWINGS_CURRENT','CNT_DRAWINGS_OTHER_CURRENT','CNT_DRAWINGS_POS_CURRENT']].fillna(0)
credit_card_balance['AMT_DRAWINGS'] = credit_card_balance['AMT_DRAWINGS_ATM_CURRENT']+credit_card_balance['AMT_DRAWINGS_CURRENT']+credit_card_balance['AMT_DRAWINGS_OTHER_CURRENT']+credit_card_balance['AMT_DRAWINGS_POS_CURRENT']
credit_card_balance['CNT_DRAWINGS'] = credit_card_balance['CNT_DRAWINGS_ATM_CURRENT']+credit_card_balance['CNT_DRAWINGS_CURRENT']+credit_card_balance['CNT_DRAWINGS_OTHER_CURRENT']+credit_card_balance['CNT_DRAWINGS_POS_CURRENT']
print(credit_card_balance)

SK_ID_PREV                         0
SK_ID_CURR                         0
MONTHS_BALANCE                     0
AMT_BALANCE                        0
AMT_CREDIT_LIMIT_ACTUAL            0
AMT_DRAWINGS_ATM_CURRENT      749816
AMT_DRAWINGS_CURRENT               0
AMT_DRAWINGS_OTHER_CURRENT    749816
AMT_DRAWINGS_POS_CURRENT      749816
AMT_INST_MIN_REGULARITY       305236
AMT_PAYMENT_CURRENT           767988
AMT_PAYMENT_TOTAL_CURRENT          0
AMT_RECEIVABLE_PRINCIPAL           0
AMT_RECIVABLE                      0
AMT_TOTAL_RECEIVABLE               0
CNT_DRAWINGS_ATM_CURRENT      749816
CNT_DRAWINGS_CURRENT               0
CNT_DRAWINGS_OTHER_CURRENT    749816
CNT_DRAWINGS_POS_CURRENT      749816
CNT_INSTALMENT_MATURE_CUM     305236
NAME_CONTRACT_STATUS               0
SK_DPD                             0
SK_DPD_DEF                         0
dtype: int64
         SK_ID_PREV  SK_ID_CURR  MONTHS_BALANCE  AMT_BALANCE  \
0           2562384      378907              -6       56.970   
1       

In [ ]:
credit_card_balance_sorted = pd.DataFrame()
credit_card_balance_sorted = credit_card_balance.groupby('SK_ID_PREV').max('MONTHS_BALANCE')[['SK_ID_CURR','AMT_BALANCE']]
credit_card_balance_sorted['OCCURENCES']=credit_card_balance.groupby('SK_ID_PREV').count()['MONTHS_BALANCE']
credit_card_balance_sorted['AVG_AMT_BALANCE'] = credit_card_balance.groupby('SK_ID_PREV')['AMT_BALANCE'].sum()/credit_card_balance.groupby('SK_ID_PREV')['AMT_BALANCE'].count()
credit_card_balance_sorted['AVG_AMT_CREDIT_LIMIT_ACTUAL'] = credit_card_balance.groupby('SK_ID_PREV')['AMT_CREDIT_LIMIT_ACTUAL'].sum()/credit_card_balance.groupby('SK_ID_PREV')['AMT_CREDIT_LIMIT_ACTUAL'].count()
credit_card_balance_sorted['AVG_AMT_DRAWINGS'] = credit_card_balance.groupby('SK_ID_PREV')['AMT_DRAWINGS'].sum()/credit_card_balance.groupby('SK_ID_PREV')['AMT_DRAWINGS'].count()
credit_card_balance_sorted['AVG_AMT_INST_MIN_REGULARITY'] = credit_card_balance.groupby('SK_ID_PREV')['AMT_INST_MIN_REGULARITY'].sum()/credit_card_balance.groupby('SK_ID_PREV')['AMT_INST_MIN_REGULARITY'].count()
credit_card_balance_sorted['AVG_AMT_PAYMENT_CURRENT'] = credit_card_balance.groupby('SK_ID_PREV')['AMT_PAYMENT_CURRENT'].sum()/credit_card_balance.groupby('SK_ID_PREV')['AMT_PAYMENT_CURRENT'].count()
credit_card_balance_sorted['AVG_AMT_PAYMENT_TOTAL_CURRENT'] = credit_card_balance.groupby('SK_ID_PREV')['AMT_PAYMENT_TOTAL_CURRENT'].sum()/credit_card_balance.groupby('SK_ID_PREV')['AMT_PAYMENT_TOTAL_CURRENT'].count()
credit_card_balance_sorted['AVG_AMT_RECEIVABLE_PRINCIPAL'] = credit_card_balance.groupby('SK_ID_PREV')['AMT_RECEIVABLE_PRINCIPAL'].sum()/credit_card_balance.groupby('SK_ID_PREV')['AMT_RECEIVABLE_PRINCIPAL'].count()
credit_card_balance_sorted['AVG_AMT_RECIVABLE'] = credit_card_balance.groupby('SK_ID_PREV')['AMT_RECIVABLE'].sum()/credit_card_balance.groupby('SK_ID_PREV')['AMT_RECIVABLE'].count()
credit_card_balance_sorted['AVG_AMT_TOTAL_RECEIVABLE'] = credit_card_balance.groupby('SK_ID_PREV')['AMT_TOTAL_RECEIVABLE'].sum()/credit_card_balance.groupby('SK_ID_PREV')['AMT_TOTAL_RECEIVABLE'].count()
credit_card_balance_sorted['AVG_CNT_DRAWINGS'] = credit_card_balance.groupby('SK_ID_PREV')['CNT_DRAWINGS'].sum()/credit_card_balance.groupby('SK_ID_PREV')['CNT_DRAWINGS'].count()
credit_card_balance_sorted['AVG_CNT_INSTALMENT_MATURE_CUM'] = credit_card_balance.groupby('SK_ID_PREV')['CNT_INSTALMENT_MATURE_CUM'].sum()/credit_card_balance.groupby('SK_ID_PREV')['CNT_INSTALMENT_MATURE_CUM'].count()
credit_card_balance_sorted['SUM_SK_DPD'] = credit_card_balance.groupby('SK_ID_PREV')['SK_DPD'].sum()
credit_card_balance_sorted['AVG_SK_DPD'] = credit_card_balance_sorted['SUM_SK_DPD']/credit_card_balance_sorted['OCCURENCES']
credit_card_balance_sorted['SUM_SK_DPD_DEF'] = credit_card_balance.groupby('SK_ID_PREV')['SK_DPD_DEF'].sum()
credit_card_balance_sorted['AVG_SK_DPD_DEF'] = credit_card_balance_sorted['SUM_SK_DPD_DEF']/credit_card_balance_sorted['OCCURENCES']
credit_card_balance_sorted['COUNT_DEMAND']=credit_card_balance[credit_card_balance['NAME_CONTRACT_STATUS']=='Demand'].groupby('SK_ID_PREV')['NAME_CONTRACT_STATUS'].count()
credit_card_balance_sorted['COUNT_COMPLETED']=credit_card_balance[credit_card_balance['NAME_CONTRACT_STATUS']=='Completed'].groupby('SK_ID_PREV')['NAME_CONTRACT_STATUS'].count()
credit_card_balance_sorted['COUNT_SIGNED']=credit_card_balance[credit_card_balance['NAME_CONTRACT_STATUS']=='Signed'].groupby('SK_ID_PREV')['NAME_CONTRACT_STATUS'].count()
credit_card_balance_sorted['COUNT_ACTIVE']=credit_card_balance[credit_card_balance['NAME_CONTRACT_STATUS']=='Active'].groupby('SK_ID_PREV')['NAME_CONTRACT_STATUS'].count()
credit_card_balance_sorted['COUNT_SENT']=credit_card_balance[credit_card_balance['NAME_CONTRACT_STATUS']=='Sent proposal'].groupby('SK_ID_PREV')['NAME_CONTRACT_STATUS'].count()
credit_card_balance_sorted['COUNT_APPROVED']=credit_card_balance[credit_card_balance['NAME_CONTRACT_STATUS']=='Approved'].groupby('SK_ID_PREV')['NAME_CONTRACT_STATUS'].count()
credit_card_balance_sorted['COUNT_REFUSED']=credit_card_balance[credit_card_balance['NAME_CONTRACT_STATUS']=='Refused'].groupby('SK_ID_PREV')['NAME_CONTRACT_STATUS'].count()
credit_card_balance_sorted[['COUNT_DEMAND','COUNT_COMPLETED','COUNT_SIGNED','COUNT_ACTIVE','COUNT_SENT',
                         'COUNT_APPROVED','COUNT_REFUSED']]=credit_card_balance_sorted[['COUNT_DEMAND','COUNT_COMPLETED','COUNT_SIGNED','COUNT_ACTIVE','COUNT_SENT',
                         'COUNT_APPROVED','COUNT_REFUSED']].fillna(0)
print(credit_card_balance_sorted)
credit_card_balance_ID_CURR = credit_card_balance_sorted.groupby('SK_ID_CURR').sum()

            SK_ID_CURR  AMT_BALANCE  OCCURENCES  AVG_AMT_BALANCE  \
SK_ID_PREV                                                         
1000018         394447   136695.420           5     74946.285000   
1000030         361282   103027.275           8     55991.064375   
1000031         131335   154945.935          16     52394.439375   
1000035         436351        0.000           5         0.000000   
1000077         181153        0.000          11         0.000000   
...                ...          ...         ...              ...   
2843476         197090   173458.800          95     37937.812263   
2843477         168439    63850.950          85      1663.076647   
2843478         424526    77646.555          90      5111.405000   
2843493         337804    78839.595          15     59139.927000   
2843496         425374   189972.450          15     44373.681000   

            AVG_AMT_CREDIT_LIMIT_ACTUAL  AVG_AMT_DRAWINGS  \
SK_ID_PREV                                            

In [ ]:
credit_card_balance_ID_CURR__application_train = credit_card_balance_ID_CURR.merge(application_train, on='SK_ID_CURR', how='inner')
print(credit_card_balance_ID_CURR__application_train)

       SK_ID_CURR  AMT_BALANCE  OCCURENCES  AVG_AMT_BALANCE  \
0          100006        0.000           6         0.000000   
1          100011   189000.000          74     54482.111149   
2          100021        0.000          17         0.000000   
3          100023        0.000           8         0.000000   
4          100036        0.000          12         0.000000   
...           ...          ...         ...              ...   
86900      456242   217274.805           8    148232.328750   
86901      456244   453627.675          41    131834.730732   
86902      456246    43490.115           8     13136.731875   
86903      456247   190202.130          95     23216.396211   
86904      456248        0.000          23         0.000000   

       AVG_AMT_CREDIT_LIMIT_ACTUAL  AVG_AMT_DRAWINGS  \
0                    270000.000000          0.000000   
1                    164189.189189       4864.864865   
2                    675000.000000          0.000000   
3                  

In [ ]:
corr = credit_card_balance_ID_CURR__application_train.corr(method='spearman')
print(corr)

                               SK_ID_CURR  AMT_BALANCE  OCCURENCES  \
SK_ID_CURR                       1.000000     0.004895   -0.001816   
AMT_BALANCE                      0.004895     1.000000    0.231363   
OCCURENCES                      -0.001816     0.231363    1.000000   
AVG_AMT_BALANCE                  0.005446     0.945569    0.141923   
AVG_AMT_CREDIT_LIMIT_ACTUAL      0.004915     0.426130   -0.193994   
AVG_AMT_DRAWINGS                 0.007179     0.849432   -0.082360   
AVG_AMT_INST_MIN_REGULARITY      0.005481     0.938516    0.214509   
AVG_AMT_PAYMENT_CURRENT          0.008149     0.858992    0.055953   
AVG_AMT_PAYMENT_TOTAL_CURRENT    0.008618     0.848252    0.102010   
AVG_AMT_RECEIVABLE_PRINCIPAL     0.005166     0.944319    0.139891   
AVG_AMT_RECIVABLE                0.005086     0.943225    0.138655   
AVG_AMT_TOTAL_RECEIVABLE         0.005070     0.943223    0.138725   
AVG_CNT_DRAWINGS                 0.007621     0.776676   -0.080058   
AVG_CNT_INSTALMENT_M

AMT_BALANCE chosen due to being relatively high correlation, high correlation to other columns with relatively high correlation, and no missing data points

In [ ]:
corr.to_excel(r'spearman.xlsx',index=True,header=True)